### Building on top of these concepts:

If you haven't covered langmem and basic openai-agents-sdk go through these notebooks first:

1. [Understand LangMem Core APIs](https://colab.research.google.com/drive/1YJNrnQRMgeNTigIuWOfykt-Z5L_DDmsa?usp=sharing)

2. [Use LangMem Memory Tools with OpenAI Agents SDK](https://colab.research.google.com/drive/1xgSUeJPIBKyjpM868PsvmCZaCof-s2vB?usp=sharing)

3. [Use Persistence Storage](https://colab.research.google.com/drive/1gA9r_FkbHFCWlgd52qz1oTW736gD0kmN?usp=sharing)

4. [Baseline Email Assistant - Prep before Memory](https://colab.research.google.com/drive/1AgedinzRuoow3f2cvR0wRlATk8AgE0x0?usp=sharing)

# **Semantic Memory**

Semantic memory stores the essential facts and other information that ground an agent's responses. Two common representations of semantic memory are collections (to record an unbounded amount of knowledge to be searched at runtime) and profiles (to record task-specific information that follows a strict schema that is easily looked up by user or agent).

### **When to Use Semantic Memories**
Semantic memories help agents learn from conversations. They extract and store meaningful information that might be useful in future interactions.

For example, when discussing a project, the agent might remember technical requirements, team structure, or key decisions - anything that could provide helpful context later.

The goal is to build understanding over time, just like humans do through repeated interactions. Not everything needs to be remembered - focus on information that helps the agent be more helpful in future conversations. Semantic memory works best when the agent is able to save important memories and the dense relationships between them so that it can later recall not just "what" but "why" and "how".

## Install Packages

In [1]:
!pip install -Uq openai-agents langmem langchain-google-genai langmem_adapter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 943.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")

# 1. **Semantic Memories Extraction**

In [4]:
from langmem import create_memory_manager #
from pydantic import BaseModel

class Triple(BaseModel): #
    """Store all new facts, preferences, and relationships as triples."""
    subject: str
    predicate: str
    object: str
    context: str | None = None

# Configure extraction
manager = create_memory_manager(
    "google_genai:gemini-2.0-flash",
    schemas=[Triple],
    instructions="Extract user preferences and any other useful information",
    enable_inserts=True,
    enable_deletes=True,
)

In [5]:
# First conversation - extract triples
conversation1 = [
    {"role": "user", "content": "We are building AI Agents to make Mars next humans stop"}
]
memories = manager.invoke({"messages": conversation1})
print("After first conversation:")
for m in memories:
    print(m)

After first conversation:
ExtractedMemory(id='5217563d-22ec-4506-9d70-efd59ac00998', content=Triple(subject='AI Agents', predicate='purpose', object='make Mars next humans stop', context=None))


The second conversation updates some existing memories. Since we have enabled "deletes", the manager will return RemoveDoc objects to indicate that the memory should be removed, and a new memory will be created in its place. Since this uses the core "functional" API (aka, it doesn't read or write to a database), you can control what "removal" means, be that a soft or hard delete, or simply a down-weighting of the memory.

In [6]:
# Second conversation - update and add triples
conversation2 = [
    {"role": "user", "content": "Junaid AI Agents Workspace can now suggest designs for AI Agents Core."}
]
update = manager.invoke({"messages": conversation2, "existing": memories})
print("After second conversation:")
for m in update:
    print(m)

existing = [m for m in update if isinstance(m.content, Triple)]

After second conversation:
ExtractedMemory(id='5217563d-22ec-4506-9d70-efd59ac00998', content=Triple(subject='AI Agents', predicate='purpose', object='suggest designs for AI Agents Core', context=None))


# 2. **OpenAI Agents SDK Email Assistant with Semantic Memory**

We'll give the assistant the ability to remember details from previous emails.

- Uses OpenAI Agents SDK
- Classifies incoming messages (respond, ignore, notify)
- Drafts responses
- Schedules meetings
- Ability to Remember details from previous Examples

We'll give the assistant the ability to remember details from previous emails.

# Memory Layer in OpenAI Agents SDK

In [8]:
from pydantic import BaseModel, Field
from agents import (
    Agent,
    Runner,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)
from typing import Dict, Any
from typing_extensions import TypedDict, Literal, Annotated


In [9]:
#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=os.environ["GOOGLE_API_KEY"],
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)


## **Setup a Profile, Prompt Instructions and Example Email**

In [10]:
profile = {
    "name": "Junaid",
    "full_name": "Muhammad Junaid Shaukat",
    "user_profile_background": "AI Products Manager Building My AI Agents Workforce.",
}

In [11]:
prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage Junaid's tasks efficiently."
}

In [12]:
# Example incoming email
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "Muhammad Junaid Shaukat<mr.junaidshaukat@gmail.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi Junaid,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

In [13]:
class Email(BaseModel):
    from_: str = Field(alias="from")
    to: str
    subject: str
    body: str

email_model = Email(**email)
print(email_model.model_dump_json(by_alias=True))

{"from":"Alice Smith <alice.smith@company.com>","to":"Muhammad Junaid Shaukat<mr.junaidshaukat@gmail.com>","subject":"Quick question about API documentation","body":"\nHi Junaid,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice"}


In [14]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

## **Prompt Templates**

In [15]:
# Triage system prompt template
triage_system_prompt_template = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Background >
{user_profile_background}.
</ Background >

< Instructions >

{name} gets lots of emails. Your job is to categorize each email into one of three categories:

1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but doesn't require a response
3. RESPOND - Emails that need a direct response from {name}

Classify the below email into one of these categories.

</ Instructions >

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email response. For these, you should notify {name} (using the `notify` response). Examples of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >
{examples}
</ Few shot examples >
"""





# Triage User Prompt Template
triage_user_prompt_template = """
Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}"""





## **A function to create a prompt using f-string**

In [16]:
def create_prompt(template: str, variables: Dict[str, any]) -> str:
    """Creates a prompt using an f-string and a dictionary of variables."""
    try:
        return template.format(**variables)
    except KeyError as e:
        return f"Error: Missing variable '{e.args[0]}' in the provided dictionary."

In [17]:
system_prompt = create_prompt(triage_system_prompt_template, {
    "full_name": profile["full_name"],
    "name":profile["name"],
    "examples": None,
    "user_profile_background": profile["user_profile_background"],
    "triage_no" : prompt_instructions["triage_rules"]["ignore"],
    "triage_notify": prompt_instructions["triage_rules"]["notify"],
    "triage_email" : prompt_instructions["triage_rules"]["respond"],
  }
)

In [18]:
user_prompt = create_prompt(triage_user_prompt_template, {
    "author": email["from"],
    "to": email["to"],
    "subject": email["subject"],
    "email_thread" : email["body"],
  }
)

## **Triage Agent**

In [19]:
triage_agent = Agent(
    name="Triage Agent",
    instructions=system_prompt,
    output_type=Router
)

In [20]:
triage_result = await Runner.run(triage_agent, user_prompt, run_config = config)
print(triage_result.final_output.classification)
print(triage_result.final_output.reasoning)


respond
Alice is asking a direct question about the API documentation. This requires a response from Junaid to clarify the missing endpoints.


## **Response Agent, Define Tools**

In [21]:
from agents import function_tool

In [22]:
@function_tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'"

In [23]:
@function_tool
def schedule_meeting(
    attendees: list[str],
    subject: str,
    duration_minutes: int,
    preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"



In [24]:
@function_tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

### **LangMem Memory Management Tools**

In [25]:
import asyncio
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langgraph.store.memory import InMemoryStore
from contextlib import asynccontextmanager

store = InMemoryStore(
      index={
          "dims": 768,
          "embed": GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
      }
    )
@asynccontextmanager
async def get_store():
  yield store


In [26]:
class UserInfo(BaseModel):
  username: str

namespace_template=("email_assistant", "{username}", "collection")

In [27]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langmem_adapter import LangMemOpenAIAgentToolAdapter  # import from your package

# Initialize the manage memory tool dynamically:
manage_adapter = LangMemOpenAIAgentToolAdapter(
    lambda store, namespace=None: create_manage_memory_tool(namespace=namespace, store=store),
    store_provider=get_store,
    namespace_template=namespace_template
)
manage_memory_tool = manage_adapter.as_tool()

# Initialize the search memory tool dynamically:
search_adapter = LangMemOpenAIAgentToolAdapter(
    lambda store, namespace=None: create_search_memory_tool(namespace=namespace, store=store),
    store_provider=get_store,
    namespace_template=namespace_template
)
search_memory_tool = search_adapter.as_tool()


## **Response Agent, Define Prompt**

In [28]:
response_prompt_template = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

In [29]:
response_system_prompt = create_prompt(response_prompt_template, {
    "full_name": profile["full_name"],
    "name":profile["name"],
    "instructions": prompt_instructions["agent_instructions"] + "Always save my email interactions in memory store for later discussions.",
  }
)
print(response_system_prompt)


< Role >
You are Muhammad Junaid Shaukat's executive assistant. You are a top-notch executive assistant who cares about Junaid performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage Junaid's communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
</ Tools >

< Instructions >
Use these tools when appropriate to help manage Junaid's tasks efficiently.Always save my email interactions in memory store for later discussions.
</ Instructions >



In [30]:
tools=[write_email, schedule_meeting, check_calendar_availability, manage_memory_tool, search_memory_tool]

In [31]:
response_agent = Agent[UserInfo](
    name="Response agent",
    instructions=response_system_prompt,
    tools=tools
    )


In [32]:
response_result = await Runner.run(response_agent,
                                   "Jim is my friend",
                                   run_config = config,
                                   context=UserInfo(username=profile["name"])
                                   )
print(response_result.final_output)


Okay, I've saved that Jim is your friend. I'll keep that in mind. Anything else I can help you with, Junaid?



In [33]:
response_result = await Runner.run(response_agent,
                                   "Do I know Jim?",
                                   run_config = config,
                                   context=UserInfo(username=profile["name"])
                                   )
print(response_result.final_output)


Yes, you know Jim. He is your friend.



In [34]:
async with get_store() as store:
  namespace=("email_assistant", profile["name"], "collection")
  res = await store.asearch(namespace)
  print(res)

[Item(namespace=['email_assistant', 'Junaid', 'collection'], key='596888b6-8151-4b70-a3aa-fcebd3907468', value={'content': 'Jim is my friend'}, created_at='2025-03-31T12:41:06.970357+00:00', updated_at='2025-03-31T12:41:06.970364+00:00', score=None)]


## **Create the Overall Flow in Python**

In [35]:
async def triage_router(email: Email, username: str):

  user_prompt = create_prompt(triage_user_prompt_template, {
    "author": email.from_,
    "to": email.to,
    "subject": email.subject,
    "email_thread" : email.body,
  })

  # print(user_prompt)

  triage_result = await Runner.run(
      triage_agent,
      user_prompt,
      run_config = config,
      context=UserInfo(username=username)
      )
  print(triage_result.final_output)
  print("Triage History: ", triage_result.to_input_list())

  if triage_result.final_output.classification == "respond":
        print("📧 Classification: RESPOND - This email requires a response")
        response_result = await Runner.run(
            response_agent,
            f"Respond to the email {email.model_dump_json(by_alias=True)}",
            run_config = config,
            context=UserInfo(username=username)
            )
        print(response_result.final_output)
        print("Response History", response_result.to_input_list())
  elif triage_result.final_output.classification == "ignore":
      print("🚫 Classification: IGNORE - This email can be safely ignored")
  elif triage_result.final_output.classification == "notify":
      # If real life, this would do something else
      print("🔔 Classification: NOTIFY - This email contains important information")
  else:
      raise ValueError(f"Invalid classification: {triage_result.final_output.classification}")


## **Now Test the Triage and Response Agents Working Together**

In [36]:
email_input = {
    "from": "Marketing Team <marketing@amazingdeals.com>",
    "to": "Muhammad Junaid Shaukat<mr.junaidshaukat@gmail.com>",
    "subject": "🔥 EXCLUSIVE OFFER: Limited Time Discount on Developer Tools! 🔥",
    "body": """Dear Valued Developer,

Don't miss out on this INCREDIBLE opportunity!

🚀 For a LIMITED TIME ONLY, get 80% OFF on our Premium Developer Suite!

✨ FEATURES:
- Revolutionary AI-powered code completion
- Cloud-based development environment
- 24/7 customer support
- And much more!

💰 Regular Price: $999/month
🎉 YOUR SPECIAL PRICE: Just $199/month!

🕒 Hurry! This offer expires in:
24 HOURS ONLY!

Click here to claim your discount: https://amazingdeals.com/special-offer

Best regards,
Marketing Team
---
To unsubscribe, click here
""",
}

typed_email = Email(**email_input)
print(typed_email.model_dump_json(by_alias=True))

{"from":"Marketing Team <marketing@amazingdeals.com>","to":"Muhammad Junaid Shaukat<mr.junaidshaukat@gmail.com>","subject":"🔥 EXCLUSIVE OFFER: Limited Time Discount on Developer Tools! 🔥","body":"Dear Valued Developer,\n\nDon't miss out on this INCREDIBLE opportunity!\n\n🚀 For a LIMITED TIME ONLY, get 80% OFF on our Premium Developer Suite!\n\n✨ FEATURES:\n- Revolutionary AI-powered code completion\n- Cloud-based development environment\n- 24/7 customer support\n- And much more!\n\n💰 Regular Price: $999/month\n🎉 YOUR SPECIAL PRICE: Just $199/month!\n\n🕒 Hurry! This offer expires in:\n24 HOURS ONLY!\n\nClick here to claim your discount: https://amazingdeals.com/special-offer\n\nBest regards,\nMarketing Team\n---\nTo unsubscribe, click here\n"}


In [37]:
await triage_router(typed_email, "Junaid")

reasoning='This is a marketing email with a promotional offer, which falls under the category of emails that do not require a response.' classification='ignore'
Triage History:  [{'content': "\nPlease determine how to handle the below email thread:\n\nFrom: Marketing Team <marketing@amazingdeals.com>\nTo: Muhammad Junaid Shaukat<mr.junaidshaukat@gmail.com>\nSubject: 🔥 EXCLUSIVE OFFER: Limited Time Discount on Developer Tools! 🔥\nDear Valued Developer,\n\nDon't miss out on this INCREDIBLE opportunity!\n\n🚀 For a LIMITED TIME ONLY, get 80% OFF on our Premium Developer Suite!\n\n✨ FEATURES:\n- Revolutionary AI-powered code completion\n- Cloud-based development environment\n- 24/7 customer support\n- And much more!\n\n💰 Regular Price: $999/month\n🎉 YOUR SPECIAL PRICE: Just $199/month!\n\n🕒 Hurry! This offer expires in:\n24 HOURS ONLY!\n\nClick here to claim your discount: https://amazingdeals.com/special-offer\n\nBest regards,\nMarketing Team\n---\nTo unsubscribe, click here\n", 'role': 'u

In [38]:
email_input2 = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "Muhammad Junaid Shaukat<mr.junaidshaukat@gmail.com>",
    "subject": "Quick question about API documentation",
    "body": """Hi Junaid,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

typed_email2 = Email(**email_input2)

In [39]:
await triage_router(typed_email2, "Junaid")

reasoning='Alice is asking a direct question about the API documentation which is a critical part of the new authentication service. It requires a response to clarify the missing endpoints.' classification='respond'
Triage History:  [{'content': "\nPlease determine how to handle the below email thread:\n\nFrom: Alice Smith <alice.smith@company.com>\nTo: Muhammad Junaid Shaukat<mr.junaidshaukat@gmail.com>\nSubject: Quick question about API documentation\nHi Junaid,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice", 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': '{\n  "reasoning": "Alice is asking a direct question about the API documentation which is a critical part of the new authentication service. It 

In [53]:
async with get_store() as store:
  namespace=("email_assistant", "Junaid", "collection")
  res = await store.asearch(namespace)
  for i, mem in enumerate(res):
    print(i,": ", mem.value['content'])

0 :  Jim is my friend
1 :  Email from Alice Smith about missing endpoints in API documentation. Replied to Alice saying I will check with the development team.
